# Parte inicial del ETL creado para coderhouse

## Importamos las librerias necesarias

In [2]:
import requests
import pandas as pd
import utils

NameError: name 'MetaData' is not defined

Creamos el log para tener de manera visual si todo funciona

Obtenenos la api key

In [ ]:
API_KEY = 'f29d2a34d2f0fcf9db7c748ca09c7a1a'
MOVIE_ID = '16869'
# 10859
# https://api.themoviedb.org/3/movie/{MOVIE_ID}/credits?language=es-MX&api_key={API_KEY}
# https://api.themoviedb.org/3/movie/{MOVIE_ID}?language=es-MX&api_key={API_KEY}

In [ ]:
#def redshift_connection(host, port, database, user, password):
def redshift_connection():
    try:
        utils.logger.info('INFO', '"Connecting to Redshift Cluster"')
        host = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
        port = 5439
        database = 'data-engineer-database'
        user = 'diego_kisai_coderhouse'
        password = ' '
        engine = utils.create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')
        utils.logger.info("Successfully connected to Redshift cluster")
        return engine
    except ConnectionError as e:
        utils.logger.error("Failed to connect to redshift, please review your access credentials")
        raise e


In [ ]:
redshift_connection()

Hacemos la consulta y validamos que existe el id que queremos guardar

In [ ]:
credits = requests.get(f'https://api.themoviedb.org/3/movie/{MOVIE_ID}/credits?language=es-MX&api_key={API_KEY}')
movie = requests.get(f'https://api.themoviedb.org/3/movie/{MOVIE_ID}?language=es-MX&api_key={API_KEY}')
if credits.status_code and movie.status_code == 200:
  print('--ok--')
  utils.logger.info(f"se encontro pelicula para el id {MOVIE_ID}")
else:
  print('--no existe pelicula para ese id--')
  utils.logger.warning("no existe pelicula para ese id")

Modificamos lo que obtenemos para crear un dataframe

In [ ]:
# c = json.loads(credits.text)
# m = json.loads(movie.text)
#cred = pd.json_normalize(c)
movi = pd.json_normalize(movie.json())
movi = movi.explode('genres')
movi = movi.explode('spoken_languages')
movi = movi.explode('production_countries')
movi = movi.explode('production_companies')

movi['genres'] = movi['genres'].apply(lambda x: x['name'])
movi['spoken_languages'] = movi['spoken_languages'].apply(lambda x: x['name'])
movi['production_countries'] = movi['production_countries'].apply(lambda x: x['name'])
#df_crew = pd.DataFrame(cred['crew'][0])
#df_cast = pd.DataFrame(cred['cast'][0])
df_movi = pd.DataFrame(movi)


In [ ]:
df_crew

In [ ]:
df_cast

In [ ]:
movi_order = df_movi[['id',
                     'title',
                     'original_title',
                     'original_language',
                     'spoken_languages',
                     'genres',
                     'release_date',
                     'poster_path',
                     'overview',
                     'budget',
                     'revenue',
                     'runtime',
                     'tagline',
                     'production_countries',
                     ]]

In [ ]:

movi_order

In [ ]:
distinct_crew = df_crew['original_name'].nunique()
distinct_cast = df_cast['original_name'].nunique()
total = distinct_crew + distinct_cast
titulo = df_movi.at[0,'title']

In [ ]:
print(f'En la pelicula {titulo} trabajaron {total} personas, hay {distinct_cast} actores y {distinct_crew} en el staff')

In [ ]:
director_rows = df_crew[df_crew['job'].isin(['Director', 'Executive Producer', 'Producer'])]

# Agrupar por 'job' y tomar la primera fila de cada grupo
first_rows = director_rows.groupby('job').first()
first_rows